![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [32]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [33]:
filename_features = "~/Desktop/UCI HAR Dataset/features.txt"
filename_labels = "~/Desktop/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "~/Desktop/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "~/Desktop/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "~/Desktop/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "~/Desktop/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "~/Desktop/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "~/Desktop/UCI HAR Dataset/test/y_test.txt"

# Reading data
# Remove `squeeze=True` and use `.iloc` to extract the Series.
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")['nome_var']
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])['subject_id']
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None)
y_train = pd.read_csv(filename_ytrain, header=None, names=['activity_id'])['activity_id']

# Test data
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None)
y_test = pd.read_csv(filename_ytest, header=None, names=['activity_id'])['activity_id']


/tmp/ipykernel_38084/863137963.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
/tmp/ipykernel_38084/863137963.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None)
/tmp/ipykernel_38084/863137963.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None)


A base original vem dividdida em treino e testes. Vamos dividir a base de treino em Treino e Validação, para podermos fazer o post-prunning da nossa árvore - e avaliar o modelo final em uma base de testes isenta.

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [45]:
%%time
import time
start_time = time.time()

# Create and fit the DecisionTreeClassifier
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf.fit(X_train, y_train)

# Evaluate accuracy
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, clf.predict(X_test))

# Print the accuracies
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Measure and print the time taken
end_time = time.time()
print(f"Time taken: {end_time - start_time:.4f} seconds")

Training Accuracy: 0.9788
Testing Accuracy: 0.8500
Time taken: 2.9700 seconds
CPU times: user 2.97 s, sys: 997 μs, total: 2.97 s
Wall time: 2.97 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [53]:
%%time
from sklearn.metrics import accuracy_score

y_pred = [0, 9, 7, 8, 8]
y_true = [0, 8, 5, 0, 0]

accuracy_score(y_true, y_pred)

accuracy_score(y_true, y_pred, normalize=False)

CPU times: user 1.01 ms, sys: 0 ns, total: 1.01 ms
Wall time: 937 μs


1.0

In [55]:
import time

acuracy_score = time.time()

end_time = time.time()
print(f"Tempo de processamento: {end_time - start_time:.4f} segundos")

Tempo de processamento: 42.1945 segundos


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [61]:
%%time

import pandas as pd
import time
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Number of components to test
components_list = [1, 2, 5, 10, 50]
results = []

for n_components in components_list:
    start_time = time.time()

    # Apply PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train DecisionTreeClassifier
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train_pca, y_train)

    # Evaluate Accuracy
    train_accuracy = accuracy_score(y_train, clf.predict(X_train_pca))
    test_accuracy = accuracy_score(y_test, clf.predict(X_test_pca))


CPU times: user 38.7 s, sys: 7.6 ms, total: 38.7 s
Wall time: 2.73 s


In [62]:
# End timing
end_time = time.time()
processing_time = end_time - start_time

# Store results
results.append({
    "Components": n_components,
    "Train Accuracy": train_accuracy,
    "Test Accuracy": test_accuracy,
    "Processing Time (s)": processing_time
})

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
print(results_df)

   Components  Train Accuracy  Test Accuracy  Processing Time (s)
0          50             1.0       0.801154            13.389539


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

a curracia de treino teve uma melhora significa que ficou em 1.0
ja na acuracia test ficou em 0.80


tempo de processamento dos dados teve ficou em 13.38 segundo ficou bem rapido nosso
modelo preditivo